In [1]:
from opencage.geocoder import OpenCageGeocode
import pandas as pd

In [2]:
key = 'ec835d3be64547f8bb252358d2b4518d'
geocoder = OpenCageGeocode(key)

In [3]:
def do_it(address):
    results = geocoder.geocode(address)
    try:
        county = results[0]['components']['county'][:-7].strip().upper()
    except KeyError:
        county = results[1]['components']['county'][:-7].strip().upper()
    except:
        print('Could not find county name for',address)
        county = ''
    return([[results[0]['geometry']['lat'],results[0]['geometry']['lng']],county])
    # returns a list in which list[0] is a list of coordinates and list[1] is the county name

In [4]:
def expand_address(row):
    if str(row['Address']) == 'nan': 
        return row['City'] + ',' + 'WI' + ' ' + row['Zip']
    else:
        return str(row['Address']) + ',' + row['City'] + ',' + 'WI' + ' ' + row['Zip']

In [5]:
def county(row):
    return row['Coordinates'][1]

In [6]:
infiles = ['../data/DFI_co-ops_complete.csv','../data/additions_to_DFI.csv']
#infiles = ['../data/test_DFI.csv','../data/test_additions.csv']
co_ops_df = pd.DataFrame()

for file in infiles:
    df = pd.read_csv(file,dtype=object)
    df['Zip'] = df['Zip'].apply(lambda z : z[0:5])
    df['Full_address'] = df.apply(expand_address,axis=1)
    df['Coordinates'] = df['Full_address'].apply(do_it)
    co_ops_df = pd.concat([co_ops_df,df], ignore_index=True, sort=False)

In [7]:
# Extract the county name from the Coordinates column and overwrite the County column with the result.
co_ops_df['County'] = co_ops_df.apply(county,axis=1)

# Remove the county name from the Coordinates column
co_ops_df['Coordinates'] = co_ops_df['Coordinates'].apply(lambda li : li[0])

In [8]:
len(co_ops_df)

759

In [9]:
outfile = "../data/co-ops_2019_all_sources.csv"
co_ops_df.to_csv(outfile,index=None)